In [68]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KernelDensity
import time      
import pandas as pd

In [69]:

def plot_kde(values, feature_name, sampler):
        kde = KernelDensity(bandwidth='silverman', kernel='gaussian')
        # plot it
        kde.fit(values[:, None])
        min = np.min(values)
        max = np.max(values)
        x = np.linspace(min, max, 100000)
        logprob = kde.score_samples(x[:, None])
        plt.fill_between(x, np.exp(logprob), alpha=0.5)
        plt.title(f'{feature_name} KDE, sampler: {sampler}')
        plt.show()



def plot_comparative_kde(values_original, values_sampled, feature_name, sampler):
    # the same but compare to kde of values_original
    kde = KernelDensity(bandwidth='silverman', kernel='gaussian')
    # plot it
    kde.fit(values_original[:, None])
    min_original = np.min(values_original)
    max_original = np.max(values_original)
    min_sampled = np.min(values_sampled)
    max_sampled = np.max(values_sampled)
    min = np.min([min_original, min_sampled])
    max = np.max([max_original, max_sampled])
                                                
    x = np.linspace(min, max, 100000)
    logprob = kde.score_samples(x[:, None])
    plt.fill_between(x, np.exp(logprob), alpha=0.5, label='original')
    kde.fit(values_sampled[:, None])

    x = np.linspace(min, max, 100000)
    logprob = kde.score_samples(x[:, None])
    plt.fill_between(x, np.exp(logprob), alpha=0.5, label='sampled')
    plt.title(f'{feature_name} KDE, sampler: {sampler}')
    plt.legend()
    plt.show()

In [70]:
def plot_kde_per_feature_sampled(paths, feature_names, sampler_names):
    dfs = [pd.read_csv(path, header=None) for path in paths]

    for i in range(dfs[0].shape[1]):
        for j in range(len(dfs)):
            df = dfs[j]
            plot_kde(df.iloc[:, i].values, feature_names[i], sampler_names[j])


def plot_comparative_kde_per_feature_sampled(original_path, paths, feature_names, sampler_names):
    original = pd.read_csv(original_path)
    dfs = [pd.read_csv(path, header=None) for path in paths]

    for i in range(original.shape[1]):
        for j in range(len(dfs)):
            df = dfs[j]
            plot_comparative_kde(original.iloc[:, i].values, df.iloc[:, i].values, feature_names[i], sampler_names[j])

In [71]:
def get_feature_names(path):
    df = pd.read_csv(path)
    return df.columns

In [72]:
DATASET_USED = 'Abalone'
SAMPLERS = ['random', 'kl']
SAMPLE_SIZE = 500

In [73]:
PATHS = [f'data/sampled_datasets/{DATASET_USED}/{sampler}/{SAMPLE_SIZE}/sample_0.csv' for sampler in SAMPLERS]
PATH_ORIGINAL = f'data/split_datasets/{DATASET_USED}/train.csv'

feature_names = get_feature_names(PATH_ORIGINAL)
print(feature_names)
#plot_kde_per_feature_sampled(PATHS, feature_names, SAMPLERS)

plot_comparative_kde_per_feature_sampled(PATH_ORIGINAL, PATHS, feature_names, SAMPLERS)